In [51]:
import os
from Bio import SeqIO
import pandas as pd
import matplotlib.pyplot as plt
from Bio.Data import CodonTable
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import seaborn as sns
from tqdm import tqdm  
from collections import Counter

### Overall Codon Counts and Codon Location

In [52]:
# Define the folder where the GenBank files are located
genbank_folder = "/Users/annasve/Desktop/data/ncbi_genomes/all_genbank/Streptomyces/"
# Define the folder where the results will be saved
output_folder = "/Users/annasve/Desktop/data/ncbi_genomes/analysis/codon_distribution_and_location/Streptomyces/"

# Define the output files
overall_codon_count_file = os.path.join(output_folder, "overall_codon_counts.csv")
strain_codon_count_file = os.path.join(output_folder, "strain_codon_counts.csv")
codon_position_file = os.path.join(output_folder, "codon_positions_normalized.csv")

# Initialize codon count dictionary, excluding start and stop codons
valid_stop_codons = {"TAA", "TAG", "TGA"}
valid_start_codons = {"ATG"}  # Start codon

# Generate all possible codons, excluding start and stop codons
codons = [
    f"{x}{y}{z}" 
    for x in "ATGC" 
    for y in "ATGC" 
    for z in "ATGC" 
    if f"{x}{y}{z}" not in valid_start_codons and f"{x}{y}{z}" not in valid_stop_codons
]

# Exclude start and stop codons from the overall codon count
overall_codon_counts = {codon: 0 for codon in codons}

# Initialize strain-specific counts and positions
strain_codon_counts = {}
organism_info = {}
codon_positions = []

# Process files in batches for memory efficiency
batch_size = 10
file_list = [f for f in os.listdir(genbank_folder) if f.endswith(".gbk") or f.endswith(".gbff")]
num_batches = len(file_list) // batch_size + (1 if len(file_list) % batch_size != 0 else 0)

for batch_idx in tqdm(range(num_batches), desc="Processing Batches", unit="batch"):
    start = batch_idx * batch_size
    end = min(start + batch_size, len(file_list))
    batch_files = file_list[start:end]
    
    for file_name in batch_files:
        strain = file_name.split('.')[0]  # Use file name as strain ID
        file_path = os.path.join(genbank_folder, file_name)
        strain_codon_counts[strain] = {codon: 0 for codon in codons}

        try:
            with open(file_path, "r") as file:
                for record in SeqIO.parse(file, "genbank"):
                    # Get organism name
                    organism = record.annotations.get("organism", "Unknown Organism")
                    organism_info[strain] = organism

                    for feature in record.features:
                        if feature.type == "CDS":
                            try:
                                # Extract the sequence
                                sequence = feature.location.extract(record.seq)
                                codon_start = int(feature.qualifiers.get("codon_start", [1])[0]) - 1
                                coding_sequence = sequence[codon_start:]
                                coding_sequence_str = str(coding_sequence).upper()
                                
                                # Exclude sequences not divisible by 3
                                if len(coding_sequence_str) % 3 != 0:
                                    continue  # Skip this sequence

                                # Break into codons
                                codons_in_gene = [
                                    coding_sequence_str[i:i+3]
                                    for i in range(0, len(coding_sequence_str), 3)
                                ]
                                
                                # Read the first codon (start codon) but do not count it
                                start_codon = codons_in_gene.pop(0)

                                # Stop counting at the first stop codon
                                if any(stop_codon in codons_in_gene for stop_codon in valid_stop_codons):
                                    stop_index = next(i for i, codon in enumerate(codons_in_gene) if codon in valid_stop_codons)
                                    codons_in_gene = codons_in_gene[:stop_index]

                                # Process each codon (excluding start and stop codons)
                                gene_length = len(codons_in_gene)
                                for i, codon in enumerate(codons_in_gene):
                                    if codon in overall_codon_counts:
                                        overall_codon_counts[codon] += 1
                                        strain_codon_counts[strain][codon] += 1
                                        relative_position = (i / gene_length) * 100 if gene_length > 0 else 0
                                        codon_positions.append({
                                            "Strain": strain,
                                            "Gene_ID": feature.qualifiers.get("locus_tag", ["unknown"])[0],
                                            "Codon": codon,
                                            "Relative_Position": relative_position
                                        })
                            except Exception as feature_error:
                                print(f"Error processing feature in {strain}: {feature_error}")
                                continue
        except Exception as file_error:
            print(f"Error processing file {file_name}: {file_error}")

        # Save periodically to manage memory
        if len(codon_positions) >= 10000:
            pd.DataFrame(codon_positions).to_csv(codon_position_file, mode='a', index=False, header=not os.path.exists(codon_position_file))
            codon_positions = []

# Save remaining codon positions
if codon_positions:
    pd.DataFrame(codon_positions).to_csv(codon_position_file, mode='a', index=False, header=not os.path.exists(codon_position_file))

# Create a DataFrame for overall codon counts
total_codons = sum(overall_codon_counts.values())
overall_codon_count_data = [
    {"Codon": codon, "Count": count, "Relative_Representation": (count / total_codons) * 100}
    for codon, count in overall_codon_counts.items()
]
pd.DataFrame(overall_codon_count_data).to_csv(overall_codon_count_file, index=False)

# Create a DataFrame for strain-specific codon counts
strain_codon_count_data = []
for strain, codon_dict in strain_codon_counts.items():
    total_codons = sum(codon_dict.values())
    organism = organism_info.get(strain, "Unknown Organism")
    for codon, count in codon_dict.items():
        strain_codon_count_data.append({
            "GCF_ID": strain,
            "Organism": organism,
            "Codon": codon,
            "Count": count,
            "Relative_Representation (%)": (count / total_codons * 100) if total_codons > 0 else 0
        })
pd.DataFrame(strain_codon_count_data).to_csv(strain_codon_count_file, index=False)


Processing Batches: 100%|████████████████| 115/115 [1:19:21<00:00, 41.41s/batch]


### Read strain_codon_count data

In [53]:
file_path =  "/Users/annasve/Desktop/data/ncbi_genomes/analysis/codon_distribution_and_location/Streptomyces/strain_codon_counts.csv"
strain_count_data = pd.read_csv(file_path)

In [54]:
# Remove square brackets from strain names in the "Organism" column
strain_count_data["Organism"] = strain_count_data["Organism"].str.replace(r"[\[\]]", "", regex=True)

# Group organisms by the first two words in the "Organism" column
strain_count_data["Strain"] = strain_count_data["Organism"].apply(lambda x: " ".join(x.split()[:2]))

In [55]:
# Create a pivot DataFrame: strains as rows, codons as columns
pivot_df = strain_count_data.pivot_table(
    index="GCF_ID",  # Rows: Strains and Organisms
    columns="Codon",  # Columns: Codons
    values="Relative_Representation (%)",  # Values: Codon counts
    aggfunc="sum"  # Aggregate by sum if multiple entries
).fillna(0)  # Fill NaN with 0 for codons not present in a strain

# Save the pivot DataFrame to a CSV file
pivot_file = os.path.join(output_folder, "pivot_codon_counts_Streptomyces.csv")
pivot_df.to_csv(pivot_file)

print(f"Pivoted codon counts saved to {pivot_file}")

Pivoted codon counts saved to /Users/annasve/Desktop/data/ncbi_genomes/analysis/codon_distribution_and_location/Streptomyces/pivot_codon_counts_Streptomyces.csv
